# Text classification with SVM

In [1]:
from sklearn.datasets import load_files
dataset = load_files('12-movie_reviews/txt_sentoken/', shuffle = False)
dataset.target_names

['neg', 'pos']

In [2]:
dataset.filenames

array(['12-movie_reviews/txt_sentoken/neg\\cv000_29416.txt',
       '12-movie_reviews/txt_sentoken/neg\\cv001_19502.txt',
       '12-movie_reviews/txt_sentoken/neg\\cv002_17424.txt', ...,
       '12-movie_reviews/txt_sentoken/pos\\cv997_5046.txt',
       '12-movie_reviews/txt_sentoken/pos\\cv998_14111.txt',
       '12-movie_reviews/txt_sentoken/pos\\cv999_13106.txt'], 
      dtype='<U49')

In [3]:
len(dataset.data)

2000

### Split data into train and test

In [4]:
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(dataset.data,
                                                         dataset.target,
                                                         test_size = 0.25,
                                                         random_state = None)

### Build a vectorizer / classifier pipeline that filters out tokens that are too rare or too frequent

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC, NuSVC

pipeline = Pipeline([
    ('vect', TfidfVectorizer(min_df = 3, max_df = 0.95)),
    ('clf', LinearSVC(C = 1000)),
])

### Build a grid search to find out whether unigrams or bigrams are more useful.
### Fit the pipeline on the training set using grid search for the parameters

In [6]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
}
grid_search = GridSearchCV(pipeline, parameters, n_jobs = -1)
grid_search.fit(docs_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

### Print the mean and std for each candidate along with the parameter settings for all the candidates explored by grid search

In [10]:
n_candidates = len(grid_search.cv_results_['params'])
for i in range(n_candidates):
    print(i, 'params - %s; mean - %0.2f; std - %0.2f' % (grid_search.cv_results_['params'][i],
                                                        grid_search.cv_results_['mean_test_score'][i],
                                                        grid_search.cv_results_['std_test_score'][i]))

0 params - {'vect__ngram_range': (1, 1)}; mean - 0.82; std - 0.02
1 params - {'vect__ngram_range': (1, 2)}; mean - 0.84; std - 0.01


### Predict the outcome on the testing set and store it in a variable

In [14]:
y_predicted = grid_search.predict(docs_test)

### Print the classification report

In [15]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_predicted,
                                   target_names = dataset.target_names))

             precision    recall  f1-score   support

        neg       0.84      0.82      0.83       251
        pos       0.83      0.85      0.84       249

avg / total       0.84      0.84      0.84       500



### Print and plot and confusion matrix

In [16]:
cm = metrics.confusion_matrix(y_test, y_predicted)
print(cm)

[[207  44]
 [ 38 211]]
